In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import demoji

# Baixe recursos adicionais do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
df = pd.read_csv('data/bolsonaro_tweets.csv')
df.head()

,date,text,likes,retweets,link
0,2020-10-21,- Não se justifica um bilionário aporte financ...,16890,2519,https://twitter.com/jairbolsonaro/status/13189...
1,2020-10-21,A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...,23349,4174,https://twitter.com/jairbolsonaro/status/13189...
2,2020-10-21,@secomvc @ItamaratyGovBr @MinEconomia @ernesto...,2874,154,https://twitter.com/jairbolsonaro/status/13188...
3,2020-10-21,- Recebi a visita do Conselheiro de Segurança ...,19593,3391,https://twitter.com/jairbolsonaro/status/13188...
4,2020-10-20,"- CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...",16221,3227,https://twitter.com/jairbolsonaro/status/13185...


In [3]:
# Verificando se há valores nulos
print(df.isnull().sum())
# Verificando o tipo de dado de cada coluna
print(df.dtypes)
# Descrição do dataset
print(df.shape)

date        7225
text           0
likes          0
retweets       0
link           0
dtype: int64
date        object
text        object
likes        int64
retweets     int64
link        object
dtype: object
(9945, 5)


In [4]:
# Converter a coluna 'date' para o tipo datetime
df['date'] = pd.to_datetime(df['date'])
# Preencher com a data anterior
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].ffill() # forward fill

print(df.isnull().sum())


date        0
text        0
likes       0
retweets    0
link        0
dtype: int64


In [5]:
# Remover os links dos tweets
def remove_links(text):
    text = re.sub(r'http\S+', '', text)
    return text

# Aplique a função para remover os links
df['text'] = df['text'].apply(lambda x: remove_links(x))

In [6]:
# Remover os likes e retweets com valor zero
df = df[df['likes'] != 0]
df = df[df['retweets'] != 0]

In [7]:
# Tokenizar os tweets

# Tokenização e pré-processamento
stop_words = set(stopwords.words('portuguese'))
ps = PorterStemmer()

def preprocess_text(text):
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    tokens = [ps.stem(token) for token in tokens]
    return tokens

# Aplicar a função de pré-processamento aos tweets
df['tokens'] = df['text'].apply(preprocess_text)

# Visualizar o resultado
print(df[['text', 'tokens']].head())

                                                text  \
0  - Não se justifica um bilionário aporte financ...   
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...   
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...   
3  - Recebi a visita do Conselheiro de Segurança ...   
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...   

                                              tokens  
0  [justifica, bilionário, aport, financeiro, med...  
1  [vacina, chinesa, joão, doria, governo, qualqu...  
2  [secomvc, itamaratygovbr, mineconomia, ernesto...  
3  [recebi, visita, conselheiro, segurança, eua, ...  
4  [curso, técnico, portaria, 524mec, 30set2020, ...  


In [8]:
# Extrair menções usando regex
df['mencoes'] = df['text'].apply(lambda x: re.findall(r'@(\w+)', x))

# Visualizar o resultado
print(df[['text', 'mencoes']].head())

                                                text  \
0  - Não se justifica um bilionário aporte financ...   
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...   
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...   
3  - Recebi a visita do Conselheiro de Segurança ...   
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...   

                                             mencoes  
0                                                 []  
1                                                 []  
2  [secomvc, ItamaratyGovBr, MinEconomia, ernesto...  
3                                                 []  
4                     [MEC_Comunicacao, mribeiroMEC]  


In [9]:
# Função para extrair e contar hashtags de um tweet
def extrai_hashtags(text):
    hashtags = re.findall(r'#(\w+)', text)
    return hashtags

# Criar uma nova coluna 'hashtags' no DataFrame
df['hashtags'] = df['text'].apply(extrai_hashtags)

# Visualizar o resultado
print(df[['text', 'hashtags']].head())

                                                text hashtags
0  - Não se justifica um bilionário aporte financ...       []
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...       []
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...       []
3  - Recebi a visita do Conselheiro de Segurança ...       []
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...       []


In [10]:
# Função para extrair emojis de um texto
def extract_emojis(text):
    return demoji.findall(text)

# Criar uma nova coluna 'emojis' no DataFrame
df['emojis'] = df['text'].apply(extract_emojis)

# Modificar a coluna para ser apenas uma lista de emojis
df['emojis'] = df['emojis'].apply(lambda x: list(x.keys()))

# Visualizar o resultado
print(df[['text', 'emojis']].head())

                                                text emojis
0  - Não se justifica um bilionário aporte financ...     []
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...     []
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...    [🤝]
3  - Recebi a visita do Conselheiro de Segurança ...     []
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...     []


In [11]:
df.head()

,date,text,likes,retweets,link,tokens,mencoes,hashtags,emojis
0,2020-10-21,- Não se justifica um bilionário aporte financ...,16890,2519,https://twitter.com/jairbolsonaro/status/13189...,"[justifica, bilionário, aport, financeiro, med...",[],[],[]
1,2020-10-21,A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...,23349,4174,https://twitter.com/jairbolsonaro/status/13189...,"[vacina, chinesa, joão, doria, governo, qualqu...",[],[],[]
2,2020-10-21,@secomvc @ItamaratyGovBr @MinEconomia @ernesto...,2874,154,https://twitter.com/jairbolsonaro/status/13188...,"[secomvc, itamaratygovbr, mineconomia, ernesto...","[secomvc, ItamaratyGovBr, MinEconomia, ernesto...",[],[🤝]
3,2020-10-21,- Recebi a visita do Conselheiro de Segurança ...,19593,3391,https://twitter.com/jairbolsonaro/status/13188...,"[recebi, visita, conselheiro, segurança, eua, ...",[],[],[]
4,2020-10-20,"- CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...",16221,3227,https://twitter.com/jairbolsonaro/status/13185...,"[curso, técnico, portaria, 524mec, 30set2020, ...","[MEC_Comunicacao, mribeiroMEC]",[],[]


In [12]:
# Ver as 25 menções mais frequentes
mencoes = df['mencoes'].explode().value_counts() # explode() cria uma linha para cada menção
print(mencoes[:30])
# Ministerios mais mencionados: MinInfraestrutura (Tarcísio), MinEconomia (Guedes), MinSaúde (Mandetta/Teich/Pazuello)
# Filhos mais mencionados: Carlos, Eduardo, Flávio 

mencoes
govbr              196
MInfraestrutura    194
BolsonaroSP        165
tarcisiogdf        147
CarlosBolsonaro    131
MinEconomia        126
FlavioBolsonaro    118
minsaude           111
mdregional_br      107
secomvc             96
DNIToficial         68
exercitooficial     67
MEC_Comunicacao     65
DefesaGovBr         50
jairbolsonaro       46
rogeriosmarinho     46
MinCidadania        45
TerezaCrisMS        44
SF_Moro             42
tvbrasilgov         42
ernestofaraujo      41
JusticaGovBR        41
PRFBrasil           40
DamaresAlves        39
mctic               38
ItamaratyGovBr      37
portosdobrasil      37
mmeioambiente       35
OdeCarvalho         35
Astro_Pontes        34
Name: count, dtype: int64


In [13]:
# HashTags mais frequentes
hashtags = df['hashtags'].explode().value_counts()
print(hashtags[:10]) # Destaque para hashtags de apoio a si mesmo e de afrontração a opositores

hashtags
choramais               10
fakenews                 5
NasRuasComBolsonaro      4
tbt                      3
FORAPT                   3
BolsonaroNaBand          3
ForaForoDeSãoPaulo       3
BRASILNÃOÉCUBA           2
JairMessiasBolsonaro     2
ESCOLASEMPARTIDO         2
Name: count, dtype: int64


In [14]:
# Emojis mais frequentes
emojis = df['emojis'].explode().value_counts()
print(emojis[:10]) # Destaque para emojis com a bandeira do Brasil e de comunicação "informal", destaque para a maior frequencia para a bandeira dos EUA

emojis
🇧🇷    702
👍🏻    389
👍     290
🤝     143
👉🏻     47
😂      46
👍🏼     22
🇺🇸     18
🤔      17
🇮🇱     16
Name: count, dtype: int64


In [15]:
# Exportar o DataFrame para um arquivo CSV e selecionar as colunas de interesse
df[['date', 'tokens','likes', 'retweets', 'mencoes', 'hashtags', 'emojis']].to_csv('data/bolsonaro_tweets_preprocessado.csv', index=False)